In [7]:
import pandas as pd
import numpy as np
import pickle as pk
import os
import sys

from rdkit import Chem

from datetime import datetime
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from xenonpy.descriptor import Fingerprints
import xenonpy
xenonpy.__version__

from tqdm.autonotebook import tqdm
from radonpy.core import poly, calc, const
from radonpy.ff.gaff2 import GAFF2
from radonpy.ff.descriptor import FF_descriptor
const.print_level = 1


/home/lulab/mambaforge/envs/ML_poly/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Prepare cyclic SMILES for polymers

The sample codes below demonstrate how to use radonpy to produce SMILES that represents a polymer that undergoes the following two operations:
1. Make N copies of the repeating unit of a polymer and connect them sequentially (head to tail) that results in a long polymer chain
2. Connect the head and tail of the long polymer chain to make a cyclic polymer used to mimic an infinitely long polymer chain

Note that the original polymer SMILES is assumed to have exactly two '*'s that represent the head and tail of a polymer

In [9]:
N_cyclic = 10 # number of repeating unit for the long polymer chain

smis_single = ['*C(CC)CC*', '*c1ccc(C*)cc1'] # two examples of polymer SMILES

smis_cyclic = [poly.make_cyclicpolymer(x, n=N_cyclic) for x in smis_single]


### Calculate descriptors for polymers and solvents

#### data preparation

Load data: in our data file, SMILES of polymers and solvents are connected with '_' initially. 

In [5]:
data = pd.read_csv('demo_data/demo_smiles.csv', index_col=0)
smis_poly = []
smis_solv = []
for smi in data['ps_pair'].values:
    tmp = smi.split('_')
    smis_poly.append(tmp[0])
    smis_solv.append(tmp[1])
    

In [1]:
import pandas as pd

# Manually enter polymer and solvent SMILES strings
polymer_smiles = ['C1=CC(=CC=C1C(=O)O)C(=O)O.C(CO)O', 'C1=CC(=CC=C1C(=O)O)C(=O)O.C(CO)O', 'C1=CC(=CC=C1C(=O)O)C(=O)O.C(CO)O']
solvent_smiles = ['[Br]Br', '[H]/C(Cl)=C(Cl)\Cl', '[H][C]([H])([N+]([O-])=O)[H]']
ps_pair = ['C1=CC(=CC=C1C(=O)O)C(=O)O.C(CO)O_[Br]Br', 'C1=CC(=CC=C1C(=O)O)C(=O)O.C(CO)O_[H]/C(Cl)=C(Cl)\Cl', 'C1=CC(=CC=C1C(=O)O)C(=O)O.C(CO)O_[H][C]([H])([N+]([O-])=O)[H]']
temp = ['100.0', '100.0', '100.0']
# Create a dataframe with the SMILES strings
data = pd.DataFrame({'Polymer': polymer_smiles, 'Solvent': solvent_smiles, 'ps_pair': ps_pair, 'temp': temp})

# Save the dataframe to a CSV file
data.to_csv('demo_data/demo_smiles.csv', index=True)

In [8]:
# extract the unique SMILES of polymers and solvents
uni_poly = np.unique(smis_poly)
uni_solv = np.unique(smis_solv)

print(f'Unique number of polymer SMILES: {uni_poly.shape[0]}')
print(f'Unique number of solvent SMILES: {uni_solv.shape[0]}')


Unique number of polymer SMILES: 1
Unique number of solvent SMILES: 3


To save time, we only calculate the unique SMILES and the assemble the descriptor for each polymer-solvent pair afterward.

In [9]:
# set up a dictionary for descriptor calculation
uni_smis = {'Polymer': uni_poly, 'Solvent': uni_solv}

# set up a dictionary to store all descriptors
desc_data = {}


Note that the final SMILES that we are using for the descriptor calculation contains 'H's explicitly. If the SMILES you have (for both the cyclic polymers and solvents) does not have explicit 'H' representation, please use the follow sample code to modify the SMILES.

In [13]:
smis_noHs = ['CCl', 'ClCCl']
smis_addHs = [Chem.MolToSmiles(Chem.AddHs(Chem.MolFromSmiles(x))) for x in smis_noHs]


#### calculate Force-field descriptors using radonpy

In [10]:
# parameters for force-field descriptors
nk = 20
sigma = 1/nk/2

for key, val in uni_smis.items():
    try:
        ff = GAFF2()
        ff_desc = FF_descriptor(ff, polar=True)
        desc_names = ff_desc.ffkm_desc_names(nk=nk)

        desc = ff_desc.ffkm_mp(list(val), nk=nk, s=sigma, cyclic=0)
            
        desc_data[f'{key}_ff'] = pd.DataFrame(desc, columns=[f'{key}_{x}' for x in desc_names], index=val)
        
        print(datetime.now())
        print(f'{key} done')
        
    except:
        print(f'{key} failed')
        pass
    
print('All done!')


2024-08-27 00:54:59.499304
Polymer done
2024-08-27 00:55:01.892503
Solvent done
All done!


#### calculate descriptors from rdkit using xenonpy

In [11]:
%%time

print(datetime.now())
print('Program started...')
for key, val in uni_smis.items():
    mols = [Chem.MolFromSmiles(x) for x in val]
    
    desc_data[f'{key}_rdk'] = Fingerprints(featurizers = 'DescriptorFeature', input_type='mol', on_errors='nan').transform(mols)
    desc_data[f'{key}_rdk']['Ipc'] = np.log(desc_data[f'{key}_rdk']['Ipc'])
    desc_data[f'{key}_rdk'].index = val
    desc_data[f'{key}_rdk'].columns = [f'{key}_{x}' for x in desc_data[f'{key}_rdk'].columns]

    print(datetime.now())
    print(f'{key} done')


2024-08-27 00:55:05.307669
Program started...


[00:55:09] DEPRECATION WARNING: please use MorganGenerator
[00:55:09] DEPRECATION WARNING: please use MorganGenerator
[00:55:09] DEPRECATION WARNING: please use MorganGenerator


2024-08-27 00:55:09.911009
Polymer done
2024-08-27 00:55:13.873633
Solvent done
CPU times: user 176 ms, sys: 897 ms, total: 1.07 s
Wall time: 8.57 s


[00:55:13] DEPRECATION WARNING: please use MorganGenerator
[00:55:13] DEPRECATION WARNING: please use MorganGenerator
[00:55:13] DEPRECATION WARNING: please use MorganGenerator
[00:55:13] DEPRECATION WARNING: please use MorganGenerator
[00:55:13] DEPRECATION WARNING: please use MorganGenerator
[00:55:13] DEPRECATION WARNING: please use MorganGenerator
[00:55:13] DEPRECATION WARNING: please use MorganGenerator
[00:55:13] DEPRECATION WARNING: please use MorganGenerator
[00:55:13] DEPRECATION WARNING: please use MorganGenerator


#### combine descriptors

In [12]:
# desc_final is in the same format as the descriptor dataframes that are stored in the sample_data folder
desc_final = pd.concat([desc_data['Polymer_ff'].loc[smis_poly,:].reset_index(drop=True),
                       desc_data['Polymer_rdk'].loc[smis_poly,:].reset_index(drop=True),
                       desc_data['Solvent_ff'].loc[smis_solv,:].reset_index(drop=True),
                       desc_data['Solvent_rdk'].loc[smis_solv,:].reset_index(drop=True)], axis=1)

print(desc_final)

desc_final.to_csv('demo_data.csv', index=False)

   Polymer_mass_H  Polymer_mass_C  Polymer_mass_N  Polymer_mass_O  \
0        0.518659        0.625483        0.594272        0.531635   
1        0.518659        0.625483        0.594272        0.531635   
2        0.518659        0.625483        0.594272        0.531635   

   Polymer_mass_F  Polymer_mass_P  Polymer_mass_S  Polymer_mass_Cl  \
0        0.391405        0.016476        0.010477         0.002155   
1        0.391405        0.016476        0.010477         0.002155   
2        0.391405        0.016476        0.010477         0.002155   

   Polymer_mass_Br  Polymer_mass_I  ...  Solvent_fr_sulfide  \
0     8.952481e-24    8.431623e-69  ...                   0   
1     8.952481e-24    8.431623e-69  ...                   0   
2     8.952481e-24    8.431623e-69  ...                   0   

   Solvent_fr_sulfonamd  Solvent_fr_sulfone  Solvent_fr_term_acetylene  \
0                     0                   0                          0   
1                     0                  

In [4]:
import pandas as pd

# Step 1: Read the data
file_path = 'demo_data/demo_desc.csv'
data = pd.read_csv(file_path)

# Step 2: Check for missing columns
if 'Polymer_mass_H' not in data.columns:
    print("Column 'Polymer_mass_H' is missing.")
else:
    print("Column 'Polymer_mass_H' is present.")

# Step 3: Display the first few rows to investigate the issue
print(data.head())

Column 'Polymer_mass_H' is present.
   Polymer_mass_H  Polymer_mass_C  Polymer_mass_N  Polymer_mass_O  \
0        0.518659        0.625483        0.594272        0.531635   
1        0.518659        0.625483        0.594272        0.531635   
2        0.518659        0.625483        0.594272        0.531635   

   Polymer_mass_F  Polymer_mass_P  Polymer_mass_S  Polymer_mass_Cl  \
0        0.391405        0.016476        0.010477         0.002155   
1        0.391405        0.016476        0.010477         0.002155   
2        0.391405        0.016476        0.010477         0.002155   

   Polymer_mass_Br  Polymer_mass_I  ...  Solvent_fr_sulfide  \
0     8.952481e-24    8.431623e-69  ...                   0   
1     8.952481e-24    8.431623e-69  ...                   0   
2     8.952481e-24    8.431623e-69  ...                   0   

   Solvent_fr_sulfonamd  Solvent_fr_sulfone  Solvent_fr_term_acetylene  \
0                     0                   0                          0   
1    